In [1]:
import pandas as pd
import numpy as np
import os
pd.options.display.max_columns = None

# 1) 'competitions' data

In [2]:
competitions = pd.read_json('../data/original-data/competitions.json', orient='records')

In [3]:
competitions.head()

,competition_id,season_id,country_name,competition_name,competition_gender,season_name,match_updated,match_available
0,16,4,Europe,Champions League,male,2018/2019,2020-07-29T05:00,2020-07-29T05:00
1,16,1,Europe,Champions League,male,2017/2018,2020-07-29T05:00,2020-07-29T05:00
2,16,2,Europe,Champions League,male,2016/2017,2020-08-26T12:33:15.869622,2020-07-29T05:00
3,16,27,Europe,Champions League,male,2015/2016,2020-08-26T12:33:15.869622,2020-07-29T05:00
4,16,26,Europe,Champions League,male,2014/2015,2020-08-26T12:33:15.869622,2020-07-29T05:00


In [4]:
competitions.dtypes

competition_id         int64
season_id              int64
country_name          object
competition_name      object
competition_gender    object
season_name           object
match_updated         object
match_available       object
dtype: object

The data types for 'match_updated' & 'match_available' should probably be datetime, but I won't be using those. In fact, I'll go ahead and drop all the columns I don't need...

In [5]:
selected_competitions = competitions[['competition_id', 'season_id', 'competition_name', 'season_name']]

In [6]:
selected_competitions.groupby('competition_name').count()

,competition_id,season_id,season_name
competition_name,,,
Champions League,15,15,15
FA Women's Super League,3,3,3
FIFA World Cup,1,1,1
La Liga,16,16,16
NWSL,1,1,1
Premier League,1,1,1
Women's World Cup,1,1,1


I'll only be looking at La Liga & Champions League games, so can drop records that belong to all the other competitions.

In [7]:
selected_competitions = selected_competitions.drop(selected_competitions[(
    selected_competitions['competition_name'] != 'Champions League') & 
    (selected_competitions['competition_name'] != 'La Liga')].index)

Lionel Messi signed his first professional contract on 24th June 2005, I can delete any seasons prior to 2005/06.

In [8]:
selected_competitions = selected_competitions.drop(selected_competitions[(
    selected_competitions['season_name'] == '1999/2000') | 
    (selected_competitions['season_name'] == '2003/2004') | 
    (selected_competitions['season_name'] == '2004/2005')].index)

In [9]:
selected_competitions

,competition_id,season_id,competition_name,season_name
0,16,4,Champions League,2018/2019
1,16,1,Champions League,2017/2018
2,16,2,Champions League,2016/2017
3,16,27,Champions League,2015/2016
4,16,26,Champions League,2014/2015
5,16,25,Champions League,2013/2014
6,16,24,Champions League,2012/2013
7,16,23,Champions League,2011/2012
8,16,22,Champions League,2010/2011
9,16,21,Champions League,2009/2010


There appears to be missing data for the 2005/06, 2007/08 & 2019/20 Champions League competitions, this is something I'll need to consider if comparing La Liga performance v Champions League performance at a later stage.

I may require the unique values from the 'competition_id' & 'season_id' columns when importing the 'matches' data, so will create those lists now...

In [10]:
competition_codes = list(selected_competitions['competition_id'].unique())
season_codes = list(selected_competitions['season_id'].unique())

In [11]:
competition_codes

[16, 11]

In [12]:
season_codes

[4, 1, 2, 27, 26, 25, 24, 23, 22, 21, 41, 39, 42, 40, 38]

# 2) 'matches' data

Import and combine multiple JSON files into a single dataframe using the La Liga competition code.

In [13]:
laliga_base_dir = r'C:\Users\paul_\Desktop\Ironhack\course-work\projects\ironhack-final-project\data\original-data\matches\11'

laliga_match_list = []
for file in os.listdir(laliga_base_dir):
    json_path = os.path.join(laliga_base_dir, file)
    json_data = pd.read_json(json_path, orient='records')
    laliga_match_list.append(json_data)
laliga_matches = pd.concat(laliga_match_list, axis=0, ignore_index=True)

In [14]:
laliga_matches

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,last_updated,metadata,match_week,competition_stage,stadium,referee
0,9592,2017-09-09,20:45:00.000,"{'competition_id': 11, 'country_name': 'Spain'...","{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 214, 'away_team_name': 'Espan...",5,0,available,2020-07-29T05:00,"{'data_version': '1.1.0', 'shot_fidelity_versi...",3,"{'id': 1, 'name': 'Regular Season'}","{'id': 342, 'name': 'Camp Nou', 'country': {'i...","{'id': 183, 'name': 'Jesús Gil', 'country': {'..."
1,9870,2018-04-07,20:45:00.000,"{'competition_id': 11, 'country_name': 'Spain'...","{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 205, 'away_team_name': 'Legan...",3,1,available,2020-07-29T05:00,"{'data_version': '1.0.2', 'shot_fidelity_versi...",31,"{'id': 1, 'name': 'Regular Season'}","{'id': 342, 'name': 'Camp Nou', 'country': {'i...","{'id': 215, 'name': 'Ricardo De Burgos', 'coun..."
2,9783,2018-02-04,16:15:00.000,"{'competition_id': 11, 'country_name': 'Spain'...","{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 214, 'home_team_name': 'Espan...","{'away_team_id': 217, 'away_team_name': 'Barce...",1,1,available,2020-07-29T05:00,"{'data_version': '1.0.2', 'shot_fidelity_versi...",22,"{'id': 1, 'name': 'Regular Season'}","{'id': 351, 'name': 'RCDE Stadium', 'country':...","{'id': 183, 'name': 'Jesús Gil', 'country': {'..."
3,9609,2017-09-19,22:00:00.000,"{'competition_id': 11, 'country_name': 'Spain'...","{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 322, 'away_team_name': 'Eibar...",6,1,available,2020-07-29T05:00,"{'data_version': '1.0.2', 'shot_fidelity_versi...",5,"{'id': 1, 'name': 'Regular Season'}","{'id': 342, 'name': 'Camp Nou', 'country': {'i...","{'id': 208, 'name': 'Alejandro Hernández', 'co..."
4,9700,2017-12-02,13:00:00.000,"{'competition_id': 11, 'country_name': 'Spain'...","{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 209, 'away_team_name': 'Celta...",2,2,available,2020-07-29T05:00,"{'data_version': '1.0.2', 'shot_fidelity_versi...",14,"{'id': 1, 'name': 'Regular Season'}","{'id': 342, 'name': 'Camp Nou', 'country': {'i...","{'id': 209, 'name': 'Mario Melero', 'country':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,303524,2019-12-01,21:00:00.000,"{'competition_id': 11, 'country_name': 'Spain'...","{'season_id': 42, 'season_name': '2019/2020'}","{'home_team_id': 212, 'home_team_name': 'Atlét...","{'away_team_id': 217, 'away_team_name': 'Barce...",0,1,available,2020-07-29T05:00,"{'data_version': '1.1.0', 'shot_fidelity_versi...",15,"{'id': 1, 'name': 'Regular Season'}","{'id': 4654, 'name': 'Estadio Wanda Metropolit...","{'id': 180, 'name': 'Antonio Mateu', 'country'..."
481,303451,2019-12-07,21:00:00.000,"{'competition_id': 11, 'country_name': 'Spain'...","{'season_id': 42, 'season_name': '2019/2020'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 1043, 'away_team_name': 'Mall...",5,2,available,2020-07-29T05:00,"{'data_version': '1.1.0', 'shot_fidelity_versi...",16,"{'id': 1, 'name': 'Regular Season'}","{'id': 342, 'name': 'Camp Nou', 'country': {'i...","{'id': 224, 'name': 'José Munuera', 'country':..."
482,303517,2019-12-21,16:00:00.000,"{'competition_id': 11, 'country_name': 'Spain'...","{'season_id': 42, 'season_name': '2019/2020'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 206, 'away_team_name': 'Depor...",4,1,available,2020-07-29T05:00,"{'data_version': '1.1.0', 'shot_fidelity_versi...",18,"{'id': 1, 'name': 'Regular Season'}","{'id': 342, 'name': 'Camp Nou', 'country': {'i...","{'id': 209, 'name': 'Mario Melero', 'country':..."
483,303682,2020-02-02,21:00:00.000,"{'competition_id

Import and combine multiple JSON files into a single dataframe using the Champions League competition code.

In [15]:
champsleague_base_dir = r'C:\Users\paul_\Desktop\Ironhack\course-work\projects\ironhack-final-project\data\original-data\matches\16'

champsleague_match_list = []
for file in os.listdir(champsleague_base_dir):
    json_path = os.path.join(champsleague_base_dir, file)
    json_data = pd.read_json(json_path, orient='records')
    champsleague_match_list.append(json_data)
champsleague_matches = pd.concat(champsleague_match_list, axis=0, ignore_index=True)

In [16]:
champsleague_matches

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,last_updated,metadata,match_week,competition_stage,stadium,referee
0,18245,2018-05-26,20:45:00.000,"{'competition_id': 16, 'country_name': 'Europe...","{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 220, 'home_team_name': 'Real ...","{'away_team_id': 24, 'away_team_name': 'Liverp...",3,1,available,2020-07-29T05:00,"{'data_version': '1.1.0', 'shot_fidelity_versi...",7,"{'id': 26, 'name': 'Final'}","{'id': 4222, 'name': 'NSK Olimpijs'kyj', 'coun...","{'id': 727, 'name': 'M. Mažić'}"
1,18244,2017-06-03,20:45:00.000,"{'competition_id': 16, 'country_name': 'Europe...","{'season_id': 2, 'season_name': '2016/2017'}","{'home_team_id': 224, 'home_team_name': 'Juven...","{'away_team_id': 220, 'away_team_name': 'Real ...",1,4,available,2020-07-29T05:00,{'data_version': '1.0.3'},7,"{'id': 26, 'name': 'Final'}","{'id': 113891, 'name': 'Principality Stadium',...","{'id': 226, 'name': 'F. Brych', 'country': {'i..."
2,18235,2010-05-22,20:45:00.000,"{'competition_id': 16, 'country_name': 'Europe...","{'season_id': 21, 'season_name': '2009/2010'}","{'home_team_id': 169, 'home_team_name': 'Bayer...","{'away_team_id': 238, 'away_team_name': 'Inter...",0,2,available,2020-07-29T05:00,{'data_version': '1.0.3'},1,"{'id': 26, 'name': 'Final'}","{'id': 353, 'name': 'Estadio Santiago Bernabéu...","{'id': 708, 'name': 'H. Webb'}"
3,18236,2011-05-28,20:45:00.000,"{'competition_id': 16, 'country_name': 'Europe...","{'season_id': 22, 'season_name': '2010/2011'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 39, 'away_team_name': 'Manche...",3,1,available,2020-07-29T05:00,{'data_version': '1.0.3'},1,"{'id': 26, 'name': 'Final'}","{'id': 4666, 'name': 'Wembley Stadium', 'count...","{'id': 449, 'name': 'V. Kassai', 'country': {'..."
4,18237,2012-05-19,20:45:00.000,"{'competition_id': 16, 'country_name': 'Europe...","{'season_id': 23, 'season_name': '2011/2012'}","{'home_team_id': 169, 'home_team_name': 'Bayer...","{'away_team_id': 33, 'away_team_name': 'Chelse...",1,1,available,2020-07-29T05:00,{'data_version': '1.0.3'},7,"{'id': 26, 'name': 'Final'}","{'id': 4867, 'name': 'Allianz Arena', 'country...","{'id': 717, 'name': 'Pedro Proença'}"
5,18240,2013-05-25,20:45:00.000,"{'competition_id': 16, 'country_name': 'Europe...","{'season_id': 24, 'season_name': '2012/2013'}","{'home_team_id': 180, 'home_team_name': 'Borus...","{'away_team_id': 169, 'away_team_name': 'Bayer...",1,2,available,2020-07-29T05:00,{'data_version': '1.0.3'},7,"{'id': 26, 'name': 'Final'}","{'id': 4666, 'name': 'Wembley Stadium', 'count...","{'id': 419, 'name': 'N. Rizzoli', 'country': {..."
6,18241,2014-05-24,20:45:00.000,"{'competition_id': 16, 'country_name': 'Europe...","{'season_id': 25, 'season_name': '2013/2014'}","{'home_team_id': 220, 'home_team_name': 'Real ...","{'away_team_id': 212, 'away_team_name': 'Atlét...",4,1,available,2020-07-29T05:00,{'data_version': '1.0.3'},7,"{'id': 26, 'name': 'Final'}","{'id': 611, 'name': 'Estádio do Sport Lisboa e...","{'id': 287, 'name': 'B. Kuipers', 'country': {..."
7,18242,2015-06-06,20:45:00.000,"{'competition_id': 16, 'country_name': 'Europe...","{'season_id': 26, 'season_name': '2014/2015'}","{'home_team_id': 224, 'home_team_name': 'Juven...","{'away_team_id': 217, 'away_team_name': 'Barce...",1,3,available,2020-07-29T05:00,{'data_version': '1.0.3'},7,"{'id': 26, 'name': 'Final'}","{'id': 367, 'name': 'Olympiastadion Berlin', '...","{'id': 581, 'name': 'C. Çakιr', 'country': {'i..."
8,18243,2016-05-28,20:45:00.000,"{'competition_id': 16, 'country_name': 'Europe...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 220, 'home_team_name': 'Real ...","{'away_team_id': 212, 'away_team_name': 'Atlét...",1,1,available,2020-07-29T05:00,{'data_version': '1.0.3'},7,"{'id': 26, 'name': 'Final'}","{'id': 388, 'name': 'Stadio Giuseppe Meazza', ...","{'id': 728, 'name': 'M. Clattenburg'}"
9,2302

The 'La Liga' dataframe consists of 485 matches, when comparing this with Lionel Messi's Wikipedia page we know this to be the amount of games that he has played for Barcelona in La Liga. 

The 'Champions League' dataframe consists of only 13 matches, these appear to the finals from each season, so probably won't be much use in our analysis.

-----------------------------------------------------
A number of columns can be dropped from the 'La Liga' dataframe...

In [17]:
laliga_matches = laliga_matches.drop(columns=['competition', 'match_status', 'last_updated', 'metadata', 'competition_stage'])

In [18]:
laliga_matches.head()

,match_id,match_date,kick_off,season,home_team,away_team,home_score,away_score,match_week,stadium,referee
0,9592,2017-09-09,20:45:00.000,"{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 214, 'away_team_name': 'Espan...",5,0,3,"{'id': 342, 'name': 'Camp Nou', 'country': {'i...","{'id': 183, 'name': 'Jesús Gil', 'country': {'..."
1,9870,2018-04-07,20:45:00.000,"{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 205, 'away_team_name': 'Legan...",3,1,31,"{'id': 342, 'name': 'Camp Nou', 'country': {'i...","{'id': 215, 'name': 'Ricardo De Burgos', 'coun..."
2,9783,2018-02-04,16:15:00.000,"{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 214, 'home_team_name': 'Espan...","{'away_team_id': 217, 'away_team_name': 'Barce...",1,1,22,"{'id': 351, 'name': 'RCDE Stadium', 'country':...","{'id': 183, 'name': 'Jesús Gil', 'country': {'..."
3,9609,2017-09-19,22:00:00.000,"{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 322, 'away_team_name': 'Eibar...",6,1,5,"{'id': 342, 'name': 'Camp Nou', 'country': {'i...","{'id': 208, 'name': 'Alejandro Hernández', 'co..."
4,9700,2017-12-02,13:00:00.000,"{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 209, 'away_team_name': 'Celta...",2,2,14,"{'id': 342, 'name': 'Camp Nou', 'country': {'i...","{'id': 209, 'name': 'Mario Melero', 'country':..."


## Splitting columns containing dictionaries...

There are a number of columns that contain dictionaries and sub-dictionaries, so I need to split these into separate columns, delete unwanted columns and then join them onto the main dataframe...

In [19]:
season_df = pd.json_normalize(laliga_matches['season'])
season_df.head()

,season_id,season_name
0,1,2017/2018
1,1,2017/2018
2,1,2017/2018
3,1,2017/2018
4,1,2017/2018


In [20]:
home_team_df = pd.json_normalize(laliga_matches['home_team'])
home_team_df = home_team_df.drop(columns=['home_team_gender', 'home_team_group', 'country.id', 'country.name'])
home_team_df.head()

,home_team_id,home_team_name,managers
0,217,Barcelona,"[{'id': 227, 'name': 'Ernesto Valverde Tejedor..."
1,217,Barcelona,"[{'id': 227, 'name': 'Ernesto Valverde Tejedor..."
2,214,Espanyol,"[{'id': 236, 'name': 'Enrique Sánchez Flores',..."
3,217,Barcelona,"[{'id': 227, 'name': 'Ernesto Valverde Tejedor..."
4,217,Barcelona,"[{'id': 227, 'name': 'Ernesto Valverde Tejedor..."


In [21]:
home_manager_df = home_team_df['managers'].apply(pd.Series).add_prefix('home_manager_')
home_manager_df.head()

,home_manager_0
0,"{'id': 227, 'name': 'Ernesto Valverde Tejedor'..."
1,"{'id': 227, 'name': 'Ernesto Valverde Tejedor'..."
2,"{'id': 236, 'name': 'Enrique Sánchez Flores', ..."
3,"{'id': 227, 'name': 'Ernesto Valverde Tejedor'..."
4,"{'id': 227, 'name': 'Ernesto Valverde Tejedor'..."


In [22]:
home_manager_df = home_manager_df['home_manager_0'].apply(pd.Series).add_prefix('home_manager_')
home_manager_df.head()

,home_manager_0,home_manager_country,home_manager_dob,home_manager_id,home_manager_name,home_manager_nickname
0,NaN,"{'id': 214, 'name': 'Spain'}",None,227.0,Ernesto Valverde Tejedor,Ernesto Valverde
1,NaN,"{'id': 214, 'name': 'Spain'}",None,227.0,Ernesto Valverde Tejedor,Ernesto Valverde
2,NaN,"{'id': 214, 'name': 'Spain'}",None,236.0,Enrique Sánchez Flores,Quique Sánchez Flores
3,NaN,"{'id': 214, 'name': 'Spain'}",None,227.0,Ernesto Valverde Tejedor,Ernesto Valverde
4,NaN,"{'id': 214, 'name': 'Spain'}",None,227.0,Ernesto Valverde Tejedor,Ernesto Valverde


In [23]:
home_manager_country_df = home_manager_df['home_manager_country'].apply(pd.Series).add_prefix('home_manager_country_')
home_manager_country_df.head()

,home_manager_country_0,home_manager_country_id,home_manager_country_name
0,NaN,214.0,Spain
1,NaN,214.0,Spain
2,NaN,214.0,Spain
3,NaN,214.0,Spain
4,NaN,214.0,Spain


In [24]:
home_team_df = home_team_df.join([home_manager_df, home_manager_country_df])

In [25]:
home_team_df.head()

,home_team_id,home_team_name,managers,home_manager_0,home_manager_country,home_manager_dob,home_manager_id,home_manager_name,home_manager_nickname,home_manager_country_0,home_manager_country_id,home_manager_country_name
0,217,Barcelona,"[{'id': 227, 'name': 'Ernesto Valverde Tejedor...",NaN,"{'id': 214, 'name': 'Spain'}",None,227.0,Ernesto Valverde Tejedor,Ernesto Valverde,NaN,214.0,Spain
1,217,Barcelona,"[{'id': 227, 'name': 'Ernesto Valverde Tejedor...",NaN,"{'id': 214, 'name': 'Spain'}",None,227.0,Ernesto Valverde Tejedor,Ernesto Valverde,NaN,214.0,Spain
2,214,Espanyol,"[{'id': 236, 'name': 'Enrique Sánchez Flores',...",NaN,"{'id': 214, 'name': 'Spain'}",None,236.0,Enrique Sánchez Flores,Quique Sánchez Flores,NaN,214.0,Spain
3,217,Barcelona,"[{'id': 227, 'name': 'Ernesto Valverde Tejedor...",NaN,"{'id': 214, 'name': 'Spain'}",None,227.0,Ernesto Valverde Tejedor,Ernesto Valverde,NaN,214.0,Spain
4,217,Barcelona,"[{'id': 227, 'name': 'Ernesto Valverde Tejedor...",NaN,"{'id': 214, 'name': 'Spain'}",None,227.0,Ernesto Valverde Tejedor,Ernesto Valverde,NaN,214.0,Spain


Drop unwanted columns created by splitting...

In [26]:
home_team_df = home_team_df.drop(columns=['managers', 'home_manager_0', 'home_manager_country', 
                                          'home_manager_dob', 'home_manager_name', 'home_manager_country_0'])

In [27]:
home_team_df.head()

,home_team_id,home_team_name,home_manager_id,home_manager_nickname,home_manager_country_id,home_manager_country_name
0,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain
1,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain
2,214,Espanyol,236.0,Quique Sánchez Flores,214.0,Spain
3,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain
4,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain


Repeat same whole process carried out on 'home_team_df' to 'away_team_df'...

In [28]:
away_team_df = pd.json_normalize(laliga_matches['away_team'])
away_team_df = away_team_df.drop(columns=['away_team_gender', 'away_team_group', 'country.id', 'country.name'])
away_manager_df = away_team_df['managers'].apply(pd.Series).add_prefix('away_manager_')
away_manager_df = away_manager_df['away_manager_0'].apply(pd.Series).add_prefix('away_manager_')
away_manager_country_df = away_manager_df['away_manager_country'].apply(pd.Series).add_prefix('away_manager_country_')
away_team_df = away_team_df.join([away_manager_df, away_manager_country_df])
away_team_df = away_team_df.drop(columns=['managers', 'away_manager_0', 'away_manager_country', 
                                          'away_manager_dob', 'away_manager_name', 'away_manager_country_0'])

In [29]:
away_team_df.head()

,away_team_id,away_team_name,away_manager_id,away_manager_nickname,away_manager_country_id,away_manager_country_name
0,214,Espanyol,236.0,Quique Sánchez Flores,214.0,Spain
1,205,Leganés,84.0,Asier Garitano,214.0,Spain
2,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain
3,322,Eibar,221.0,José Luis Mendilibar,214.0,Spain
4,209,Celta Vigo,208.0,Juan Carlos Unzué,214.0,Spain


In [30]:
stadium_df = laliga_matches['stadium'].apply(pd.Series).add_prefix('stadium_')
stadium_df = stadium_df.drop(columns=['stadium_0', 'stadium_country'])
stadium_df.head()

,stadium_id,stadium_name
0,342.0,Camp Nou
1,342.0,Camp Nou
2,351.0,RCDE Stadium
3,342.0,Camp Nou
4,342.0,Camp Nou


In [31]:
referee_df = laliga_matches['referee'].apply(pd.Series).add_prefix('referee_')
referee_df.head()

,referee_0,referee_country,referee_id,referee_name
0,NaN,"{'id': 214, 'name': 'Spain'}",183.0,Jesús Gil
1,NaN,"{'id': 112, 'name': 'Italy'}",215.0,Ricardo De Burgos
2,NaN,"{'id': 214, 'name': 'Spain'}",183.0,Jesús Gil
3,NaN,"{'id': 112, 'name': 'Italy'}",208.0,Alejandro Hernández
4,NaN,"{'id': 112, 'name': 'Italy'}",209.0,Mario Melero


In [32]:
referee_country_df = referee_df['referee_country'].apply(pd.Series).add_prefix('referee_country_')
referee_country_df.head()

,referee_country_0,referee_country_id,referee_country_name
0,NaN,214.0,Spain
1,NaN,112.0,Italy
2,NaN,214.0,Spain
3,NaN,112.0,Italy
4,NaN,112.0,Italy


In [33]:
referee_df = referee_df.join(referee_country_df)
referee_df.head()

,referee_0,referee_country,referee_id,referee_name,referee_country_0,referee_country_id,referee_country_name
0,NaN,"{'id': 214, 'name': 'Spain'}",183.0,Jesús Gil,NaN,214.0,Spain
1,NaN,"{'id': 112, 'name': 'Italy'}",215.0,Ricardo De Burgos,NaN,112.0,Italy
2,NaN,"{'id': 214, 'name': 'Spain'}",183.0,Jesús Gil,NaN,214.0,Spain
3,NaN,"{'id': 112, 'name': 'Italy'}",208.0,Alejandro Hernández,NaN,112.0,Italy
4,NaN,"{'id': 112, 'name': 'Italy'}",209.0,Mario Melero,NaN,112.0,Italy


In [34]:
referee_df = referee_df.drop(columns=['referee_0', 'referee_country', 'referee_country_0'])
referee_df.head()

,referee_id,referee_name,referee_country_id,referee_country_name
0,183.0,Jesús Gil,214.0,Spain
1,215.0,Ricardo De Burgos,112.0,Italy
2,183.0,Jesús Gil,214.0,Spain
3,208.0,Alejandro Hernández,112.0,Italy
4,209.0,Mario Melero,112.0,Italy


Join all dataframes onto 'la_liga_matches' dataframe...

In [35]:
laliga_matches = laliga_matches.join([season_df, home_team_df, away_team_df, stadium_df, referee_df])
laliga_matches.head()

,match_id,match_date,kick_off,season,home_team,away_team,home_score,away_score,match_week,stadium,referee,season_id,season_name,home_team_id,home_team_name,home_manager_id,home_manager_nickname,home_manager_country_id,home_manager_country_name,away_team_id,away_team_name,away_manager_id,away_manager_nickname,away_manager_country_id,away_manager_country_name,stadium_id,stadium_name,referee_id,referee_name,referee_country_id,referee_country_name
0,9592,2017-09-09,20:45:00.000,"{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 214, 'away_team_name': 'Espan...",5,0,3,"{'id': 342, 'name': 'Camp Nou', 'country': {'i...","{'id': 183, 'name': 'Jesús Gil', 'country': {'...",1,2017/2018,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain,214,Espanyol,236.0,Quique Sánchez Flores,214.0,Spain,342.0,Camp Nou,183.0,Jesús Gil,214.0,Spain
1,9870,2018-04-07,20:45:00.000,"{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 205, 'away_team_name': 'Legan...",3,1,31,"{'id': 342, 'name': 'Camp Nou', 'country': {'i...","{'id': 215, 'name': 'Ricardo De Burgos', 'coun...",1,2017/2018,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain,205,Leganés,84.0,Asier Garitano,214.0,Spain,342.0,Camp Nou,215.0,Ricardo De Burgos,112.0,Italy
2,9783,2018-02-04,16:15:00.000,"{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 214, 'home_team_name': 'Espan...","{'away_team_id': 217, 'away_team_name': 'Barce...",1,1,22,"{'id': 351, 'name': 'RCDE Stadium', 'country':...","{'id': 183, 'name': 'Jesús Gil', 'country': {'...",1,2017/2018,214,Espanyol,236.0,Quique Sánchez Flores,214.0,Spain,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain,351.0,RCDE Stadium,183.0,Jesús Gil,214.0,Spain
3,9609,2017-09-19,22:00:00.000,"{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 322, 'away_team_name': 'Eibar...",6,1,5,"{'id': 342, 'name': 'Camp Nou', 'country': {'i...","{'id': 208, 'name': 'Alejandro Hernández', 'co...",1,2017/2018,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain,322,Eibar,221.0,José Luis Mendilibar,214.0,Spain,342.0,Camp Nou,208.0,Alejandro Hernández,112.0,Italy
4,9700,2017-12-02,13:00:00.000,"{'season_id': 1, 'season_name': '2017/2018'}","{'home_team_id': 217, 'home_team_name': 'Barce...","{'away_team_id': 209, 'away_team_name': 'Celta...",2,2,14,"{'id': 342, 'name': 'Camp Nou', 'country': {'i...","{'id': 209, 'name': 'Mario Melero', 'country':...",1,2017/2018,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain,209,Celta Vigo,208.0,Juan Carlos Unzué,214.0,Spain,342.0,Camp Nou,209.0,Mario Melero,112.0,Italy


Drop unwanted columns...

In [36]:
laliga_matches = laliga_matches.drop(columns=['season', 'home_team', 'away_team', 'stadium', 'referee'])
laliga_matches.head()

,match_id,match_date,kick_off,home_score,away_score,match_week,season_id,season_name,home_team_id,home_team_name,home_manager_id,home_manager_nickname,home_manager_country_id,home_manager_country_name,away_team_id,away_team_name,away_manager_id,away_manager_nickname,away_manager_country_id,away_manager_country_name,stadium_id,stadium_name,referee_id,referee_name,referee_country_id,referee_country_name
0,9592,2017-09-09,20:45:00.000,5,0,3,1,2017/2018,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain,214,Espanyol,236.0,Quique Sánchez Flores,214.0,Spain,342.0,Camp Nou,183.0,Jesús Gil,214.0,Spain
1,9870,2018-04-07,20:45:00.000,3,1,31,1,2017/2018,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain,205,Leganés,84.0,Asier Garitano,214.0,Spain,342.0,Camp Nou,215.0,Ricardo De Burgos,112.0,Italy
2,9783,2018-02-04,16:15:00.000,1,1,22,1,2017/2018,214,Espanyol,236.0,Quique Sánchez Flores,214.0,Spain,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain,351.0,RCDE Stadium,183.0,Jesús Gil,214.0,Spain
3,9609,2017-09-19,22:00:00.000,6,1,5,1,2017/2018,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain,322,Eibar,221.0,José Luis Mendilibar,214.0,Spain,342.0,Camp Nou,208.0,Alejandro Hernández,112.0,Italy
4,9700,2017-12-02,13:00:00.000,2,2,14,1,2017/2018,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain,209,Celta Vigo,208.0,Juan Carlos Unzué,214.0,Spain,342.0,Camp Nou,209.0,Mario Melero,112.0,Italy


## Look into null/missing values...

In [37]:
laliga_matches.isnull().sum()

match_id                       0
match_date                     0
kick_off                       0
home_score                     0
away_score                     0
match_week                     0
season_id                      0
season_name                    0
home_team_id                   0
home_team_name                 0
home_manager_id              171
home_manager_nickname        185
home_manager_country_id      171
home_manager_country_name    171
away_team_id                   0
away_team_name                 0
away_manager_id              171
away_manager_nickname        184
away_manager_country_id      171
away_manager_country_name    171
stadium_id                     1
stadium_name                   1
referee_id                     1
referee_name                   1
referee_country_id           212
referee_country_name         212
dtype: int64

In [38]:
laliga_matches.loc[laliga_matches['stadium_id'].isna()]

,match_id,match_date,kick_off,home_score,away_score,match_week,season_id,season_name,home_team_id,home_team_name,home_manager_id,home_manager_nickname,home_manager_country_id,home_manager_country_name,away_team_id,away_team_name,away_manager_id,away_manager_nickname,away_manager_country_id,away_manager_country_name,stadium_id,stadium_name,referee_id,referee_name,referee_country_id,referee_country_name
367,16275,2019-04-23,21:30:00.000,0,2,34,4,2018/2019,206,Deportivo Alavés,187.0,Abelardo,214.0,Spain,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain,NaN,NaN,NaN,NaN,NaN,NaN


I can see that the single null values we have for 'stadium_id', 'stadium_name', 'referee_id' & 'referee_name' all belong to this one record above.

Having carried out some brief research I found out that the match was played at the 'Mendizorroza' stadium, and the referee was Adrián Cordero Vega from Spain.

So I will search through to see if either this stadium or referee appear elsewhere in the dataframe, in order to get their ID codes...

In [39]:
laliga_matches.loc[laliga_matches['stadium_name'] == 'Mendizorroza']

,match_id,match_date,kick_off,home_score,away_score,match_week,season_id,season_name,home_team_id,home_team_name,home_manager_id,home_manager_nickname,home_manager_country_id,home_manager_country_name,away_team_id,away_team_name,away_manager_id,away_manager_nickname,away_manager_country_id,away_manager_country_name,stadium_id,stadium_name,referee_id,referee_name,referee_country_id,referee_country_name


In [40]:
laliga_matches.loc[laliga_matches['home_team_name'] == 'Deportivo Alavés']

,match_id,match_date,kick_off,home_score,away_score,match_week,season_id,season_name,home_team_id,home_team_name,home_manager_id,home_manager_nickname,home_manager_country_id,home_manager_country_name,away_team_id,away_team_name,away_manager_id,away_manager_nickname,away_manager_country_id,away_manager_country_name,stadium_id,stadium_name,referee_id,referee_name,referee_country_id,referee_country_name
24,9581,2017-08-26,18:15:00.000,0,2,2,1,2017/2018,206,Deportivo Alavés,511.0,Luis Zubeldía,11.0,Argentina,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain,348.0,Estadio de Mendizorroza,223.0,Carlos Del Cerro,112.0,Italy
51,266273,2017-02-11,16:15:00.000,0,6,22,2,2016/2017,206,Deportivo Alavés,681.0,None,11.0,Argentina,217,Barcelona,793.0,Luis Enrique,214.0,Spain,348.0,Estadio de Mendizorroza,1008.0,Clos Gómez,NaN,NaN
367,16275,2019-04-23,21:30:00.000,0,2,34,4,2018/2019,206,Deportivo Alavés,187.0,Abelardo,214.0,Spain,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain,NaN,NaN,NaN,NaN,NaN,NaN
452,303421,2020-07-19,17:00:00.000,0,5,38,42,2019/2020,206,Deportivo Alavés,2.0,Juan Muñiz,214.0,Spain,217,Barcelona,238.0,Quique Setién,214.0,Spain,348.0,Estadio de Mendizorroza,207.0,Juan Martínez,112.0,Italy


In [41]:
laliga_matches.loc[laliga_matches['referee_name'] == 'Adrián Cordero Vega']

,match_id,match_date,kick_off,home_score,away_score,match_week,season_id,season_name,home_team_id,home_team_name,home_manager_id,home_manager_nickname,home_manager_country_id,home_manager_country_name,away_team_id,away_team_name,away_manager_id,away_manager_nickname,away_manager_country_id,away_manager_country_name,stadium_id,stadium_name,referee_id,referee_name,referee_country_id,referee_country_name


Based on the results from the research and searches above, we can replace the null values for 'stadium_id', 'stadium_name', & 'referee_name', but we will need to assign the referee a new 'referee_id'...

Check to see which 'referee_id' values are not in use:

In [42]:
laliga_matches["referee_id"].unique()

array([ 183.,  215.,  208.,  209.,  211.,  218.,  222.,  220.,  219.,
        180.,  223.,  216.,  224.,  217.,  221.,  207.,  210., 1008.,
       1012.,  407., 1013., 1003., 1010., 1070., 1001., 1014.,  668.,
        997., 1000., 1007., 1069., 1009., 1016., 1015.,  453.,  882.,
        993.,  994.,  995.,  996.,  998., 1002., 1005., 1006., 1011.,
         nan,  451.,  865.,  492.,  874.])

In [43]:
laliga_matches.loc[laliga_matches['home_team_name'] == 'Deportivo Alavés']

,match_id,match_date,kick_off,home_score,away_score,match_week,season_id,season_name,home_team_id,home_team_name,home_manager_id,home_manager_nickname,home_manager_country_id,home_manager_country_name,away_team_id,away_team_name,away_manager_id,away_manager_nickname,away_manager_country_id,away_manager_country_name,stadium_id,stadium_name,referee_id,referee_name,referee_country_id,referee_country_name
24,9581,2017-08-26,18:15:00.000,0,2,2,1,2017/2018,206,Deportivo Alavés,511.0,Luis Zubeldía,11.0,Argentina,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain,348.0,Estadio de Mendizorroza,223.0,Carlos Del Cerro,112.0,Italy
51,266273,2017-02-11,16:15:00.000,0,6,22,2,2016/2017,206,Deportivo Alavés,681.0,None,11.0,Argentina,217,Barcelona,793.0,Luis Enrique,214.0,Spain,348.0,Estadio de Mendizorroza,1008.0,Clos Gómez,NaN,NaN
367,16275,2019-04-23,21:30:00.000,0,2,34,4,2018/2019,206,Deportivo Alavés,187.0,Abelardo,214.0,Spain,217,Barcelona,227.0,Ernesto Valverde,214.0,Spain,NaN,NaN,NaN,NaN,NaN,NaN
452,303421,2020-07-19,17:00:00.000,0,5,38,42,2019/2020,206,Deportivo Alavés,2.0,Juan Muñiz,214.0,Spain,217,Barcelona,238.0,Quique Setién,214.0,Spain,348.0,Estadio de Mendizorroza,207.0,Juan Martínez,112.0,Italy


In [44]:
laliga_matches.loc[367, 'stadium_id'] = 348
laliga_matches.loc[367, 'stadium_name'] = 'Estadio de Mendizorroza'
laliga_matches.loc[367, 'referee_name'] = 'Adrián Cordero Vega'
laliga_matches.loc[367, 'referee_country_id'] = 214
laliga_matches.loc[367, 'referee_country_name'] = 'Spain'
laliga_matches.loc[367, 'referee_id'] = 244

Check that the fields have been correctly updated:

In [45]:
laliga_matches.iloc[367]

match_id                                       16275
match_date                                2019-04-23
kick_off                                21:30:00.000
home_score                                         0
away_score                                         2
match_week                                        34
season_id                                          4
season_name                                2018/2019
home_team_id                                     206
home_team_name                      Deportivo Alavés
home_manager_id                                  187
home_manager_nickname                       Abelardo
home_manager_country_id                          214
home_manager_country_name                      Spain
away_team_id                                     217
away_team_name                             Barcelona
away_manager_id                                  227
away_manager_nickname               Ernesto Valverde
away_manager_country_id                       

In [46]:
laliga_matches.isnull().sum()

match_id                       0
match_date                     0
kick_off                       0
home_score                     0
away_score                     0
match_week                     0
season_id                      0
season_name                    0
home_team_id                   0
home_team_name                 0
home_manager_id              171
home_manager_nickname        185
home_manager_country_id      171
home_manager_country_name    171
away_team_id                   0
away_team_name                 0
away_manager_id              171
away_manager_nickname        184
away_manager_country_id      171
away_manager_country_name    171
stadium_id                     0
stadium_name                   0
referee_id                     0
referee_name                   0
referee_country_id           211
referee_country_name         211
dtype: int64

I'm still left with several columns that have quite a high number of null values, relating to the home/away manager and referee. I could probably find out the correct values, but it would be very time consuming. 

Therefore, for now I won't use these variables in my analysis, but could revisit them if I have time in the future. I will create a subset that contains only the columns with zero null values...

In [47]:
la_liga_matches = laliga_matches[laliga_matches.columns[~laliga_matches.isnull().any()]]

## Check data types...

In [48]:
la_liga_matches.dtypes

match_id            int64
match_date         object
kick_off           object
home_score          int64
away_score          int64
match_week          int64
season_id           int64
season_name        object
home_team_id        int64
home_team_name     object
away_team_id        int64
away_team_name     object
stadium_id        float64
stadium_name       object
referee_id        float64
referee_name       object
dtype: object

'match_date' should be datetime, the format of 'kick_off' needs to be changed, plus 'stadium_id' & 'referee_id' need to be converted to int...

In [49]:
la_liga_matches['match_date'] = pd.to_datetime(la_liga_matches['match_date'], format='%Y/%m/%d')
la_liga_matches['kick_off'] = la_liga_matches['kick_off'].str[0:5]
#laliga_matches[col] = laliga_matches[col].astype('int64')
la_liga_matches = la_liga_matches.astype({'stadium_id': 'int64', 'referee_id': 'int64'})

C:\Users\paul_\anaconda3\envs\matrix\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\paul_\anaconda3\envs\matrix\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [50]:
la_liga_matches.dtypes

match_id                   int64
match_date        datetime64[ns]
kick_off                  object
home_score                 int64
away_score                 int64
match_week                 int64
season_id                  int64
season_name               object
home_team_id               int64
home_team_name            object
away_team_id               int64
away_team_name            object
stadium_id                 int64
stadium_name              object
referee_id                 int64
referee_name              object
dtype: object

In [51]:
la_liga_matches.head()

,match_id,match_date,kick_off,home_score,away_score,match_week,season_id,season_name,home_team_id,home_team_name,away_team_id,away_team_name,stadium_id,stadium_name,referee_id,referee_name
0,9592,2017-09-09,20:45,5,0,3,1,2017/2018,217,Barcelona,214,Espanyol,342,Camp Nou,183,Jesús Gil
1,9870,2018-04-07,20:45,3,1,31,1,2017/2018,217,Barcelona,205,Leganés,342,Camp Nou,215,Ricardo De Burgos
2,9783,2018-02-04,16:15,1,1,22,1,2017/2018,214,Espanyol,217,Barcelona,351,RCDE Stadium,183,Jesús Gil
3,9609,2017-09-19,22:00,6,1,5,1,2017/2018,217,Barcelona,322,Eibar,342,Camp Nou,208,Alejandro Hernández
4,9700,2017-12-02,13:00,2,2,14,1,2017/2018,217,Barcelona,209,Celta Vigo,342,Camp Nou,209,Mario Melero


## Reorder columns...

In [52]:
la_liga_matches.columns

Index(['match_id', 'match_date', 'kick_off', 'home_score', 'away_score',
       'match_week', 'season_id', 'season_name', 'home_team_id',
       'home_team_name', 'away_team_id', 'away_team_name', 'stadium_id',
       'stadium_name', 'referee_id', 'referee_name'],
      dtype='object')

In [53]:
la_liga_matches = la_liga_matches[['match_id', 'season_id', 'season_name', 'match_week', 'match_date', 'kick_off', 'home_score', 'away_score', 'home_team_id', 'home_team_name', 'away_team_id', 'away_team_name', 'stadium_id', 'stadium_name', 'referee_id', 'referee_name']]
la_liga_matches.head()

,match_id,season_id,season_name,match_week,match_date,kick_off,home_score,away_score,home_team_id,home_team_name,away_team_id,away_team_name,stadium_id,stadium_name,referee_id,referee_name
0,9592,1,2017/2018,3,2017-09-09,20:45,5,0,217,Barcelona,214,Espanyol,342,Camp Nou,183,Jesús Gil
1,9870,1,2017/2018,31,2018-04-07,20:45,3,1,217,Barcelona,205,Leganés,342,Camp Nou,215,Ricardo De Burgos
2,9783,1,2017/2018,22,2018-02-04,16:15,1,1,214,Espanyol,217,Barcelona,351,RCDE Stadium,183,Jesús Gil
3,9609,1,2017/2018,5,2017-09-19,22:00,6,1,217,Barcelona,322,Eibar,342,Camp Nou,208,Alejandro Hernández
4,9700,1,2017/2018,14,2017-12-02,13:00,2,2,217,Barcelona,209,Celta Vigo,342,Camp Nou,209,Mario Melero


I will need the unique values from the 'match_id' column when importing the 'events' data, so will create that list now...

In [54]:
match_codes = list(laliga_matches['match_id'].unique())

In [55]:
len(match_codes)

485

# 3) 'events' data

I'm expecting the following processes to take some time to run, so I'll import a module to show progress.

In [56]:
import tqdm

Import and combine multiple JSON files into a single dataframe using the 'match_codes' list created earlier.

In [57]:
events_base_dir = r'..\data\original-data\events'
events_list = []

for code in tqdm.tqdm(match_codes):
    file_dir = events_base_dir + '\\' + str(code) + '.json'
    #for file in os.listdir(events_base_dir):
        #json_path = os.path.join(events_base_dir, file)
    json_data = pd.read_json(file_dir, orient='records')
    json_data['match_id'] = code
    events_list.append(json_data)
events = pd.concat(events_list, axis=0, ignore_index=True)

100%|██████████| 485/485 [05:22<00:00,  1.51it/s]


In [58]:
events.head()

,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,team,duration,tactics,related_events,player,position,location,pass,carry,under_pressure,dribble,ball_receipt,out,clearance,duel,miscontrol,shot,goalkeeper,counterpress,ball_recovery,foul_won,off_camera,foul_committed,block,interception,substitution,bad_behaviour,match_id,50_50,injury_stoppage,half_start,player_off,half_end
0,62fcd4a2-94ed-42f0-9c4e-d994ae217440,1,1,2020-09-30 00:00:00.000,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 217, 'name': 'Barcelona'}","{'id': 1, 'name': 'Regular Play'}","{'id': 217, 'name': 'Barcelona'}",0.000,"{'formation': 442, 'lineup': [{'player': {'id'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9592,NaN,NaN,NaN,NaN,NaN
1,dcd47e2f-6a76-499e-abee-413417bd067e,2,1,2020-09-30 00:00:00.000,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 217, 'name': 'Barcelona'}","{'id': 1, 'name': 'Regular Play'}","{'id': 214, 'name': 'Espanyol'}",0.000,"{'formation': 442, 'lineup': [{'player': {'id'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9592,NaN,NaN,NaN,NaN,NaN
2,1d6e44c2-fb77-479e-bb98-851952e9b3f9,3,1,2020-09-30 00:00:00.000,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 217, 'name': 'Barcelona'}","{'id': 1, 'name': 'Regular Play'}","{'id': 214, 'name': 'Espanyol'}",0.000,NaN,[d8eaa47d-d089-45a6-b3b4-a59ab8f65a6a],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9592,NaN,NaN,NaN,NaN,NaN
3,d8eaa47d-d089-45a6-b3b4-a59ab8f65a6a,4,1,2020-09-30 00:00:00.000,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 217, 'name': 'Barcelona'}","{'id': 1, 'name': 'Regular Play'}","{'id': 217, 'name': 'Barcelona'}",0.000,NaN,[1d6e44c2-fb77-479e-bb98-851952e9b3f9],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9592,NaN,NaN,NaN,NaN,NaN
4,547584c5-1b05-46ce-b61a-cd124d20b385,5,1,2020-09-30 00:00:07.279,0,7,"{'id': 30, 'name': 'Pass'}",2,"{'id': 217, 'name': 'Barcelona'}","{'id': 9, 'name': 'From Kick Off'}","{'id': 217, 'name': 'Barcelona'}",0.806,NaN,[40265cdf-29b5-4e8a-8e9d-1d53a750ff51],"{'id': 5246, 'name': 'Luis Alberto Suárez Díaz'}","{'id': 24, 'name': 'Left Center Forward'}","[61.0, 41.0]","{'recipient': {'id': 5203, 'name': 'Sergio Bus...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9592,NaN,NaN,NaN,NaN,NaN


I'd like to subset the 'events' dataframe into separate seasons, but the dataframe only contains the 'match_id'. Therefore I will need to use this column to create a new 'season_name' column. I'll do this by subsetting the 'la_liga_matches' dataframe, and then merging with the 'events' dataframe.

In [59]:
match_season = la_liga_matches[['match_id', 'season_name']]
match_season

,match_id,season_name
0,9592,2017/2018
1,9870,2017/2018
2,9783,2017/2018
3,9609,2017/2018
4,9700,2017/2018
...,...,...
480,303524,2019/2020
481,303451,2019/2020
482,303517,2019/2020
483,303682,2019/2020


In [60]:
events = events.merge(match_season, how='left', on='match_id')

In [61]:
events.head()

,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,team,duration,tactics,related_events,player,position,location,pass,carry,under_pressure,dribble,ball_receipt,out,clearance,duel,miscontrol,shot,goalkeeper,counterpress,ball_recovery,foul_won,off_camera,foul_committed,block,interception,substitution,bad_behaviour,match_id,50_50,injury_stoppage,half_start,player_off,half_end,season_name
0,62fcd4a2-94ed-42f0-9c4e-d994ae217440,1,1,2020-09-30 00:00:00.000,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 217, 'name': 'Barcelona'}","{'id': 1, 'name': 'Regular Play'}","{'id': 217, 'name': 'Barcelona'}",0.000,"{'formation': 442, 'lineup': [{'player': {'id'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9592,NaN,NaN,NaN,NaN,NaN,2017/2018
1,dcd47e2f-6a76-499e-abee-413417bd067e,2,1,2020-09-30 00:00:00.000,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 217, 'name': 'Barcelona'}","{'id': 1, 'name': 'Regular Play'}","{'id': 214, 'name': 'Espanyol'}",0.000,"{'formation': 442, 'lineup': [{'player': {'id'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9592,NaN,NaN,NaN,NaN,NaN,2017/2018
2,1d6e44c2-fb77-479e-bb98-851952e9b3f9,3,1,2020-09-30 00:00:00.000,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 217, 'name': 'Barcelona'}","{'id': 1, 'name': 'Regular Play'}","{'id': 214, 'name': 'Espanyol'}",0.000,NaN,[d8eaa47d-d089-45a6-b3b4-a59ab8f65a6a],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9592,NaN,NaN,NaN,NaN,NaN,2017/2018
3,d8eaa47d-d089-45a6-b3b4-a59ab8f65a6a,4,1,2020-09-30 00:00:00.000,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 217, 'name': 'Barcelona'}","{'id': 1, 'name': 'Regular Play'}","{'id': 217, 'name': 'Barcelona'}",0.000,NaN,[1d6e44c2-fb77-479e-bb98-851952e9b3f9],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9592,NaN,NaN,NaN,NaN,NaN,2017/2018
4,547584c5-1b05-46ce-b61a-cd124d20b385,5,1,2020-09-30 00:00:07.279,0,7,"{'id': 30, 'name': 'Pass'}",2,"{'id': 217, 'name': 'Barcelona'}","{'id': 9, 'name': 'From Kick Off'}","{'id': 217, 'name': 'Barcelona'}",0.806,NaN,[40265cdf-29b5-4e8a-8e9d-1d53a750ff51],"{'id': 5246, 'name': 'Luis Alberto Suárez Díaz'}","{'id': 24, 'name': 'Left Center Forward'}","[61.0, 41.0]","{'recipient': {'id': 5203, 'name': 'Sergio Bus...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9592,NaN,NaN,NaN,NaN,NaN,2017/2018


# 4) 'lineups' data

To create the 'lineups' dataframe we will use the same process as with the 'events' dataframe...

In [62]:
lineups_base_dir = r'..\data\original-data\lineups'
lineups_list = []

for code in tqdm.tqdm(match_codes):
    file_dir = lineups_base_dir + '\\' + str(code) + '.json'
    #for file in os.listdir(events_base_dir):
        #json_path = os.path.join(events_base_dir, file)
    json_data = pd.read_json(file_dir, orient='records')
    json_data['match_id'] = code
    lineups_list.append(json_data)
lineups = pd.concat(lineups_list, axis=0, ignore_index=True)

100%|██████████| 485/485 [00:07<00:00, 63.97it/s]


In [63]:
lineups.head()

,team_id,team_name,lineup,match_id
0,214,Espanyol,"[{'player_id': 3236, 'player_name': 'Sergi Dar...",9592
1,217,Barcelona,"[{'player_id': 3726, 'player_name': 'Gerard De...",9592
2,205,Leganés,"[{'player_id': 3302, 'player_name': 'Nordin Am...",9870
3,217,Barcelona,"[{'player_id': 3501, 'player_name': 'Philippe ...",9870
4,214,Espanyol,"[{'player_id': 3236, 'player_name': 'Sergi Dar...",9783


In [64]:
match_season = la_liga_matches[['match_id', 'season_name']]
match_season

,match_id,season_name
0,9592,2017/2018
1,9870,2017/2018
2,9783,2017/2018
3,9609,2017/2018
4,9700,2017/2018
...,...,...
480,303524,2019/2020
481,303451,2019/2020
482,303517,2019/2020
483,303682,2019/2020


In [65]:
lineups = lineups.merge(match_season, how='left', on='match_id')

In [66]:
lineups.head()

,team_id,team_name,lineup,match_id,season_name
0,214,Espanyol,"[{'player_id': 3236, 'player_name': 'Sergi Dar...",9592,2017/2018
1,217,Barcelona,"[{'player_id': 3726, 'player_name': 'Gerard De...",9592,2017/2018
2,205,Leganés,"[{'player_id': 3302, 'player_name': 'Nordin Am...",9870,2017/2018
3,217,Barcelona,"[{'player_id': 3501, 'player_name': 'Philippe ...",9870,2017/2018
4,214,Espanyol,"[{'player_id': 3236, 'player_name': 'Sergi Dar...",9783,2017/2018


Split the 'lineups' dataframe into separate dataframes for Barcelona line-ups and opposition line-ups.

In [67]:
lineups_barcelona = lineups[lineups['team_name'] == 'Barcelona']
lineups_opposition = lineups[lineups['team_name'] != 'Barcelona']
lineups_barcelona.head()

,team_id,team_name,lineup,match_id,season_name
1,217,Barcelona,"[{'player_id': 3726, 'player_name': 'Gerard De...",9592,2017/2018
3,217,Barcelona,"[{'player_id': 3501, 'player_name': 'Philippe ...",9870,2017/2018
5,217,Barcelona,"[{'player_id': 3501, 'player_name': 'Philippe ...",9783,2017/2018
6,217,Barcelona,"[{'player_id': 3726, 'player_name': 'Gerard De...",9609,2017/2018
8,217,Barcelona,"[{'player_id': 5203, 'player_name': 'Sergio Bu...",9700,2017/2018


Similarly to the 'matches' data, the 'lineup' column contains dictionaries, so I need to split this column into separate columns, delete unwanted columns and then join them onto the main dataframe...

In [76]:
split_player_barcelona = lineups_barcelona['lineup'].apply(pd.Series).add_prefix('player_')
split_player_barcelona

,player_0,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10,player_11,player_12,player_13,player_14,player_15,player_16,player_17,player_18,player_19,player_20,player_21,player_22
1,"{'player_id': 3726, 'player_name': 'Gerard Deu...","{'player_id': 5203, 'player_name': 'Sergio Bus...","{'player_id': 5211, 'player_name': 'Jordi Alba...","{'player_id': 5213, 'player_name': 'Gerard Piq...","{'player_id': 5216, 'player_name': 'Andrés Ini...","{'player_id': 5246, 'player_name': 'Luis Alber...","{'player_id': 5470, 'player_name': 'Ivan Rakit...","{'player_id': 5477, 'player_name': 'Ousmane De...","{'player_id': 5492, 'player_name': 'Samuel Yve...","{'player_id': 5503, 'player_name': 'Lionel And...","{'player_id': 5506, 'player_name': 'Javier Ale...","{'player_id': 5542, 'player_name': 'José Paulo...","{'player_id': 6374, 'player_name': 'Nélson Cab...","{'player_id': 6402, 'player_name': 'Francisco ...","{'player_id': 6849, 'player_name': 'Lucas Dign...","{'player_id': 8652, 'player_name': 'Jasper Cil...","{'player_id': 10802, 'player_name': 'André Fil...","{'player_id': 20055, 'player_name': 'Marc-Andr...",NaN,NaN,NaN,NaN,NaN
3,"{'player_id': 3501, 'player_name': 'Philippe C...","{'player_id': 5211, 'player_name': 'Jordi Alba...","{'player_id': 5213, 'player_name': 'Gerard Piq...","{'player_id': 5216, 'player_name': 'Andrés Ini...","{'player_id': 5246, 'player_name': 'Luis Alber...","{'player_id': 5470, 'player_name': 'Ivan Rakit...","{'player_id': 5477, 'player_name': 'Ousmane De...","{'player_id': 5492, 'player_name': 'Samuel Yve...","{'player_id': 5503, 'player_name': 'Lionel And...","{'player_id': 5542, 'player_name': 'José Paulo...","{'player_id': 6332, 'player_name': 'Thomas Ver...","{'player_id': 6374, 'player_name': 'Nélson Cab...","{'player_id': 6379, 'player_name': 'Sergi Robe...","{'player_id': 6402, 'player_name': 'Francisco ...","{'player_id': 6609, 'player_name': 'Denis Suár...","{'player_id': 8652, 'player_name': 'Jasper Cil...","{'player_id': 10802, 'player_name': 'André Fil...","{'player_id': 20055, 'player_name': 'Marc-Andr...",NaN,NaN,NaN,NaN,NaN
5,"{'player_id': 3501, 'player_name': 'Philippe C...","{'player_id': 5203, 'player_name': 'Sergio Bus...","{'player_id': 5211, 'player_name': 'Jordi Alba...","{'player_id': 5213, 'player_name': 'Gerard Piq...","{'player_id': 5216, 'player_name': 'Andrés Ini...","{'player_id': 5246, 'player_name': 'Luis Alber...","{'player_id': 5492, 'player_name': 'Samuel Yve...","{'player_id': 5503, 'player_name': 'Lionel And...","{'player_id': 5542, 'player_name': 'José Paulo...","{'player_id': 6374, 'player_name': 'Nélson Cab...","{'player_id': 6379, 'player_name': 'Sergi Robe...","{'player_id': 6402, 'player_name': 'Francisco ...","{'player_id': 6849, 'player_name': 'Lucas Dign...","{'player_id': 20055, 'player_name': 'Marc-Andr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"{'player_id': 3726, 'player_name': 'Gerard Deu...","{'player_id': 5203, 'player_name': 'Sergio Bus...","{'player_id': 5211, 'player_name': 'Jordi Alba...","{'player_id': 5213, 'player_name': 'Gerard Piq...","{'player_id': 5216, 'player_name': 'Andrés Ini...","{'player_id': 5246, 'player_name': 'Luis Alber...","{'player_id': 5470, 'player_name': 'Ivan Rakit...","{'player_id': 5503, 'player_name': 'Lionel And...","{'player_id': 5506, 'player_name': 'Javier Ale...","{'player_id': 5542, 'player_name': 'José Paulo...","{'player_id': 6332, 'player_name': 'Thomas Ver...","{'player_id': 6374, 'player_name': 'Nélson Cab...","{'player_id': 6379, 'player_name': 'Sergi Robe...","{'player_id': 6400, 'player_name': 'Aleix Vida...","{'player_id': 6609, 'player_name': 'Denis Suár...","{'player_id': 6849, 'player_name': 'Lucas Dign...","{'player_id': 8652, 'player_name': 'Jasper Cil...","{'player_id': 20055, 'player_name': 'Marc-Andr...",NaN,NaN,NaN,NaN,NaN
8,"{'player_id': 5203, 'player_name': 'Sergio Bus...","{'player_id': 5211, 'player_name': 'Jordi Alba...","{'player_id': 5213, 'player_name

In [77]:
split_player_opposition = lineups_opposition['lineup'].apply(pd.Series).add_prefix('player_')
split_player_opposition

,player_0,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10,player_11,player_12,player_13,player_14,player_15,player_16,player_17,player_18,player_19,player_20,player_21,player_22
0,"{'player_id': 3236, 'player_name': 'Sergi Dard...","{'player_id': 6751, 'player_name': 'Javier Fue...","{'player_id': 6754, 'player_name': 'David Lópe...","{'player_id': 6756, 'player_name': 'Marc Navar...","{'player_id': 6757, 'player_name': 'Aarón Mart...","{'player_id': 6758, 'player_name': 'Víctor Sán...","{'player_id': 6761, 'player_name': 'Leonardo C...","{'player_id': 6763, 'player_name': 'Pablo Dani...","{'player_id': 6766, 'player_name': 'Gerard Mor...","{'player_id': 6767, 'player_name': 'Sergio Gar...","{'player_id': 6768, 'player_name': 'Diego Lópe...","{'player_id': 6769, 'player_name': 'Edinaldo G...","{'player_id': 6790, 'player_name': 'Mario Herm...","{'player_id': 6792, 'player_name': 'Pau López ...","{'player_id': 6866, 'player_name': 'Marc Roca ...","{'player_id': 6867, 'player_name': 'Papa Kouly...","{'player_id': 6870, 'player_name': 'José Manue...","{'player_id': 6886, 'player_name': 'Dídac Vilá...",NaN,NaN,NaN,NaN,NaN
2,"{'player_id': 3302, 'player_name': 'Nordin Amr...","{'player_id': 6597, 'player_name': 'Gabriel Ap...","{'player_id': 6598, 'player_name': 'Unai Busti...","{'player_id': 6599, 'player_name': 'Rubén Salv...","{'player_id': 6600, 'player_name': 'Claudio Be...","{'player_id': 6601, 'player_name': 'Raúl Garcí...","{'player_id': 6602, 'player_name': 'Miguel Áng...","{'player_id': 6603, 'player_name': 'Dimitrios ...","{'player_id': 6604, 'player_name': 'Darko Braš...","{'player_id': 6605, 'player_name': 'Joseba Zal...","{'player_id': 6606, 'player_name': 'Javier Era...","{'player_id': 6607, 'player_name': 'Julián Oma...","{'player_id': 6608, 'player_name': 'Nabil El Z...","{'player_id': 6610, 'player_name': 'Iván Cuéll...","{'player_id': 6834, 'player_name': 'Gerard Gum...","{'player_id': 6894, 'player_name': 'Ezequiel M...","{'player_id': 6914, 'player_name': 'Roberto Ro...","{'player_id': 8642, 'player_name': 'Nereo Cham...",NaN,NaN,NaN,NaN,NaN
4,"{'player_id': 3236, 'player_name': 'Sergi Dard...","{'player_id': 5685, 'player_name': 'Carlos Alb...","{'player_id': 6754, 'player_name': 'David Lópe...","{'player_id': 6756, 'player_name': 'Marc Navar...","{'player_id': 6757, 'player_name': 'Aarón Mart...","{'player_id': 6758, 'player_name': 'Víctor Sán...","{'player_id': 6761, 'player_name': 'Leonardo C...","{'player_id': 6766, 'player_name': 'Gerard Mor...","{'player_id': 6767, 'player_name': 'Sergio Gar...","{'player_id': 6768, 'player_name': 'Diego Lópe...","{'player_id': 6769, 'player_name': 'Edinaldo G...","{'player_id': 6770, 'player_name': 'Esteban Fé...","{'player_id': 6791, 'player_name': 'Javier Lóp...","{'player_id': 6870, 'player_name': 'José Manue...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"{'player_id': 5687, 'player_name': 'Takashi In...","{'player_id': 6698, 'player_name': 'Marko Dmit...","{'player_id': 6699, 'player_name': 'Ander Capa...","{'player_id': 6700, 'player_name': 'Sergio Enr...","{'player_id': 6701, 'player_name': 'Joan Jordá...","{'player_id': 6702, 'player_name': 'David Rodr...","{'player_id': 6703, 'player_name': 'Christian ...","{'player_id': 6705, 'player_name': 'José Ángel...","{'player_id': 6707, 'player_name': 'Charles Dí...","{'player_id': 6708, 'player_name': 'Paulo Andr...","{'player_id': 6709, 'player_name': 'Anaitz Arb...","{'player_id': 6710, 'player_name': 'Rubén Peña...","{'player_id': 6712, 'player_name': 'Gonzalo Es...","{'player_id': 6775, 'player_name': 'Daniel Gar...","{'player_id': 6776, 'player_name': 'David Junc...","{'player_id': 6924, 'player_name': 'Alejandro ...","{'player_id': 7900, 'player_name': 'Tiago Manu...","{'player_id': 10763, 'player_name': 'Asier Rie...",NaN,NaN,NaN,NaN,NaN
9,"{'player_id': 5217, 'player_name': 'Iago Aspas...","{'player_id': 5516, 'player_name': 'Pione Sist...","{'player_id': 6038, 'player_name

In [89]:
lineups_barcelona = pd.concat([lineups_barcelona, split_player_barcelona], axis=1).drop(columns='lineup')
lineups_opposition = pd.concat([lineups_opposition, split_player_opposition], axis=1).drop(columns='lineup')

In [91]:
lineups_barcelona

,team_id,team_name,match_id,season_name,player_0,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10,player_11,player_12,player_13,player_14,player_15,player_16,player_17,player_18,player_19,player_20,player_21,player_22
1,217,Barcelona,9592,2017/2018,"{'player_id': 3726, 'player_name': 'Gerard Deu...","{'player_id': 5203, 'player_name': 'Sergio Bus...","{'player_id': 5211, 'player_name': 'Jordi Alba...","{'player_id': 5213, 'player_name': 'Gerard Piq...","{'player_id': 5216, 'player_name': 'Andrés Ini...","{'player_id': 5246, 'player_name': 'Luis Alber...","{'player_id': 5470, 'player_name': 'Ivan Rakit...","{'player_id': 5477, 'player_name': 'Ousmane De...","{'player_id': 5492, 'player_name': 'Samuel Yve...","{'player_id': 5503, 'player_name': 'Lionel And...","{'player_id': 5506, 'player_name': 'Javier Ale...","{'player_id': 5542, 'player_name': 'José Paulo...","{'player_id': 6374, 'player_name': 'Nélson Cab...","{'player_id': 6402, 'player_name': 'Francisco ...","{'player_id': 6849, 'player_name': 'Lucas Dign...","{'player_id': 8652, 'player_name': 'Jasper Cil...","{'player_id': 10802, 'player_name': 'André Fil...","{'player_id': 20055, 'player_name': 'Marc-Andr...",NaN,NaN,NaN,NaN,NaN
3,217,Barcelona,9870,2017/2018,"{'player_id': 3501, 'player_name': 'Philippe C...","{'player_id': 5211, 'player_name': 'Jordi Alba...","{'player_id': 5213, 'player_name': 'Gerard Piq...","{'player_id': 5216, 'player_name': 'Andrés Ini...","{'player_id': 5246, 'player_name': 'Luis Alber...","{'player_id': 5470, 'player_name': 'Ivan Rakit...","{'player_id': 5477, 'player_name': 'Ousmane De...","{'player_id': 5492, 'player_name': 'Samuel Yve...","{'player_id': 5503, 'player_name': 'Lionel And...","{'player_id': 5542, 'player_name': 'José Paulo...","{'player_id': 6332, 'player_name': 'Thomas Ver...","{'player_id': 6374, 'player_name': 'Nélson Cab...","{'player_id': 6379, 'player_name': 'Sergi Robe...","{'player_id': 6402, 'player_name': 'Francisco ...","{'player_id': 6609, 'player_name': 'Denis Suár...","{'player_id': 8652, 'player_name': 'Jasper Cil...","{'player_id': 10802, 'player_name': 'André Fil...","{'player_id': 20055, 'player_name': 'Marc-Andr...",NaN,NaN,NaN,NaN,NaN
5,217,Barcelona,9783,2017/2018,"{'player_id': 3501, 'player_name': 'Philippe C...","{'player_id': 5203, 'player_name': 'Sergio Bus...","{'player_id': 5211, 'player_name': 'Jordi Alba...","{'player_id': 5213, 'player_name': 'Gerard Piq...","{'player_id': 5216, 'player_name': 'Andrés Ini...","{'player_id': 5246, 'player_name': 'Luis Alber...","{'player_id': 5492, 'player_name': 'Samuel Yve...","{'player_id': 5503, 'player_name': 'Lionel And...","{'player_id': 5542, 'player_name': 'José Paulo...","{'player_id': 6374, 'player_name': 'Nélson Cab...","{'player_id': 6379, 'player_name': 'Sergi Robe...","{'player_id': 6402, 'player_name': 'Francisco ...","{'player_id': 6849, 'player_name': 'Lucas Dign...","{'player_id': 20055, 'player_name': 'Marc-Andr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,217,Barcelona,9609,2017/2018,"{'player_id': 3726, 'player_name': 'Gerard Deu...","{'player_id': 5203, 'player_name': 'Sergio Bus...","{'player_id': 5211, 'player_name': 'Jordi Alba...","{'player_id': 5213, 'player_name': 'Gerard Piq...","{'player_id': 5216, 'player_name': 'Andrés Ini...","{'player_id': 5246, 'player_name': 'Luis Alber...","{'player_id': 5470, 'player_name': 'Ivan Rakit...","{'player_id': 5503, 'player_name': 'Lionel And...","{'player_id': 5506, 'player_name': 'Javier Ale...","{'player_id': 5542, 'player_name': 'José Paulo...","{'player_id': 6332, 'player_name': 'Thomas Ver...","{'player_id': 6374, 'player_name': 'Nélson Cab...","{'player_id': 6379, 'player_name': 'Sergi Robe...","{'player_id': 6400, 'player_name': 'Aleix Vida...","{'player_id': 6609, 'player_name': 'Denis Suár...","{'player_id': 6849, 'player_name': 'Lucas Dign...","{'player_id': 8652, 'player_name': 'Jasper Cil...","{'player_id': 20055, 'player_name': 'Marc-Andr...",NaN,

# Export created dataframes to CSV...

In [92]:
selected_competitions = selected_competitions.to_csv('../data/cleaned-data/competitions.csv', index=False)

In [93]:
la_liga_matches = la_liga_matches.to_csv('../data/cleaned-data/la_liga_matches.csv', index=False)

In [94]:
events = events.to_csv('../data/cleaned-data/events.csv', index=False)

In [95]:
lineups_barcelona = lineups_barcelona.to_csv('../data/cleaned-data/lineups_barcelona.csv', index=False)

In [96]:
lineups_opposition = lineups_opposition.to_csv('../data/cleaned-data/lineups_opposition.csv', index=False)